# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with ```mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json```

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import subprocess
import os

## Importing the Dataset
The data set can be imported using the ```mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json``` command in the Terminal Window. Instead of manually running this command in the Terminal we have opted to automate this process using ```subprocess''' module. The following program opens the terminal, goes to the secified resources folder and runs the mongoimport command to import the specified file into Mongo DB.

In [2]:


# Find the current directory
current_dir = os.getcwd()

# Loaction of the resources folder 
resources_dir = os.path.join(current_dir, "resources")

# Mongo Import command for 'esatblishment' collection.
command_list = [
    "mongoimport", 
    "--type", "json", 
    "-d", "uk_food", 
    "-c", "establishments",
    "--drop",
    "--jsonArray",
    "establishments.json" 
]

# Make resources folder the working directory
os.chdir(resources_dir)

# Running the Mongo Import command in the terminal
result = subprocess.run(command_list)

2024-04-28T20:38:10.056+0100	connected to: mongodb://localhost/
2024-04-28T20:38:10.057+0100	dropping: uk_food.establishments
2024-04-28T20:38:11.292+0100	39779 document(s) imported successfully. 0 document(s) failed to import.


In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [7]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'Reach Court Farm',
 'AddressLine2': 'Reach Road',
 'AddressLine3': 'St Margarets At Cliffe',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Barn',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.77622109052,
 'FHRSID': 429982,
 'LocalAuthorityBusinessID': 'PI/000043978',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6AQ',
 'RatingDate': '2017-09-21T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('662ea5a26ba794666faab5f8'),
 'geocode': {'latitude': '51.147379', 'longitude': '1.371115'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/429982',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate'

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data

new_restaurant = (
    {
        "BusinessName":"Penang Flavours",
        "BusinessType":"Restaurant/Cafe/Canteen",
        "BusinessTypeID":"",
        "AddressLine1":"Penang Flavours",
        "AddressLine2":"146A Plumstead Rd",
        "AddressLine3":"London",
        "AddressLine4":"",
        "PostCode":"SE18 7DY",
        "Phone":"",
        "LocalAuthorityCode":"511",
        "LocalAuthorityName":"Greenwich",
        "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
        "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
        "scores":{
            "Hygiene":"",
            "Structural":"",
            "ConfidenceInManagement":""
        },
        "SchemeType":"FHRS",
        "geocode":{
            "longitude":"0.08384000",
            "latitude":"51.49014200"
        },
        "RightToReply":"",
        "Distance":4623.9723280747176,
        "NewRatingPending":True
    }
)

In [10]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('662ea5c696121abc5f73125d'), acknowledged=True)

In [11]:
# Check that the new restaurant was inserted
query = {"BusinessName":"Penang Flavours"}
pprint(establishments.find_one(query))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('662ea5c696121abc5f73125d'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType":"Restaurant/Cafe/Canteen"}
fields = {"BusinessType":1 , "BusinessTypeID":1}

pprint(list(establishments.find(query, fields)))

[{'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('662ea5a26ba794666faab5f8')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('662ea5a26ba794666faab5f9')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('662ea5a26ba794666faab5fa')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('662ea5a26ba794666faab5fc')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('662ea5a26ba794666faab601')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('662ea5a26ba794666faab604')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('662ea5a26ba794666faab605')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('662ea5a26ba794666faab607')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_

From the above query, it is understood that food outlets with the ```BusinessType``` 'Restaurant/Cafe/Canteen' have ```BusinessTypeID``` 1

3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({'BusinessName':'Penang Flavours'}, {'$set': {'BusinessTypeID' : 1} })

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [14]:
# Confirm that the new restaurant was updated
query = {'BusinessName':'Penang Flavours'}
fields = {'BusinessName': 1,'BusinessTypeID':1 }
pprint(establishments.find_one(query, fields))

{'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 '_id': ObjectId('662ea5c696121abc5f73125d')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}
establishments.count_documents(query)

994

In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {'LocalAuthorityName': 'Dover'}
establishments.delete_many(query)

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [17]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName': 'Dover'}
establishments.count_documents(query)

0

In [18]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('662ea5a26ba794666faab8da'),
 'FHRSID': 647177,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6PY',
 'Phone': '',
 'RatingValue': '4',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingDate': '2014-03-31T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.196408', 'latitude': '51.086058'},
 'RightToReply': '',
 'Distance': 4591.821311183521,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude and latitude
# mechanics.update_many({}, [ {'$set':{ "wages.hourly_rate" : {'$toDouble': "$wages.hourly_rate"}}} ])
establishments.update_many({},[ {'$set':{'geocode.longitude' : {'$toDouble': '$geocode.longitude'}}} ])
establishments.update_many({},[ {'$set':{'geocode.latitude' : {'$toDouble': '$geocode.latitude'}}} ])

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [21]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({},[ {'$set':{'RatingValue' : {'$toInt': '$RatingValue'}}} ])

UpdateResult({'n': 38786, 'nModified': 34695, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [22]:
# Check that the coordinates and rating value are now numbers
query = {}
fields = {'_id':0,'geocode':1, 'RatingValue' :1}
establishments.find_one(query,fields)

{'RatingValue': 4, 'geocode': {'longitude': 1.196408, 'latitude': 51.086058}}